In [ ]:
import sqlite3
import pandas as pd
from pathlib import Path

from ipywidgets import (
    VBox, HBox, Button, Dropdown, Label, Output, Layout
)
from IPython.display import display, clear_output
from testresults import TestResultsAnalyzer
from CWpreprocessing import CSVtoSQLite


In [4]:
DATA_DIR = Path("CWdatafiles")
DB_PATH = "coursework.db"
CSV_PATTERN = str(DATA_DIR / "*.csv")


In [5]:
output = Output(layout={'border': '1px solid black'})


In [ ]:
def preprocess_csvs():
    with output:
        clear_output()

        csv_pattern = csv_path_input.value.strip()

        if not csv_pattern:
            print("Please provide a CSV file path or pattern.")
            return

        try:
            converter = CSVtoSQLite("dummy.csv")
            converter.convert(DB_PATH, csv_pattern=csv_pattern)
            print(f"CSV preprocessing completed for: {csv_pattern}")
        except Exception as e:
            print("Error during preprocessing:")
            print(e)



def list_tables():
    with output:
        clear_output()
        conn = sqlite3.connect(DB_PATH)
        tables = pd.read_sql(
            "SELECT name FROM sqlite_master WHERE type='table';",
            conn
        )
        conn.close()
        display(tables)


def view_table(table_name):
    with output:
        clear_output()
        conn = sqlite3.connect(DB_PATH)
        df = pd.read_sql(f"SELECT * FROM {table_name}", conn)
        conn.close()
        display(df.head())


In [7]:
def get_table_dropdown():
    conn = sqlite3.connect(DB_PATH)
    tables = pd.read_sql(
        "SELECT name FROM sqlite_master WHERE type='table';",
        conn
    )
    conn.close()

    return Dropdown(
        options=tables['name'].tolist(),
        description="Table:",
        layout=Layout(width="60%")
    )


In [8]:
btn_preprocess = Button(
    description="1️⃣ Preprocess CSV Files",
    button_style="success",
    layout=Layout(width="300px")
)

btn_list_tables = Button(
    description="2️⃣ List Database Tables",
    button_style="info",
    layout=Layout(width="300px")
)

btn_view_table = Button(
    description="3️⃣ View Selected Table",
    button_style="warning",
    layout=Layout(width="300px")
)


In [9]:
table_dropdown = get_table_dropdown()

btn_preprocess.on_click(lambda x: preprocess_csvs())
btn_list_tables.on_click(lambda x: list_tables())

def on_view_click(b):
    view_table(table_dropdown.value)

btn_view_table.on_click(on_view_click)


In [10]:
menu = VBox([
    Label("📊 Module Leader Menu"),
    btn_preprocess,
    btn_list_tables,
    table_dropdown,
    btn_view_table,
    output
])

display(menu)
